<a href="https://colab.research.google.com/github/8kta/AnatomyOfMatplotlib/blob/master/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Used to change filepaths
from pathlib import Path

# We set up matplotlib, pandas, and the display function
%matplotlib inline
import matplotlib.pyplot as plt
#import matplotlib.plt.imshow
from IPython.display import display
import pandas as pd

# import numpy to use in this cell
import numpy as np
from numpy import random


# import Image from PIL so we can use it later
from PIL import Image


import torch
import torchvision
from torchvision.utils import save_image 
import torchvision.utils

import requests

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
url = 'https://raw.githubusercontent.com/8kta/deeplearning_project/main/core/Sphere.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('Sphere.py', 'w') as f:
    f.write(r.text)

# now we can import
import Sphere

url = 'https://raw.githubusercontent.com/8kta/deeplearning_project/main/core/RandomAnchors.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('RandomAnchors.py', 'w') as f:
    f.write(r.text)

# now we can import
import RandomAnchors

url = 'https://raw.githubusercontent.com/8kta/deeplearning_project/main/core/dataset.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('dataset.py', 'w') as f:
    f.write(r.text)

# now we can import
import dataset

In [3]:
batch_size_train = 64
batch_size_test = 1000

indices = np.arange(59968)
np.random.shuffle(indices)

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

#dataset =  torchvision.datasets.MNIST('Users\8kta\Documents\GitHub\deeplearning_project', train=True, download=True,
#                             transform=torchvision.transforms.Compose([
#                               torchvision.transforms.ToTensor(),
#                               torchvision.transforms.Normalize(
#                                 (0.1307,), (0.3081,))
#                             ]))

dataset_train =  torchvision.datasets.MNIST('Users\8kta\Documents\GitHub\deeplearning_project', train=True, download=True,
                             transform=torchvision.transforms.ToTensor())
dataset_val =  torchvision.datasets.MNIST('Users\8kta\Documents\GitHub\deeplearning_project', train=True, download=True,
                             transform=torchvision.transforms.ToTensor())

dataset_test =  torchvision.datasets.MNIST('Users\8kta\Documents\GitHub\deeplearning_project', train=False, 
                                           download=True,transform=torchvision.transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size_train,drop_last=True, 
                                           sampler =torch.utils.data.SubsetRandomSampler(indices[:40000]))
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size_train,drop_last=True,
                                          sampler =torch.utils.data.SubsetRandomSampler(indices[40000:59968]))
test_loader = torch.utils.data.DataLoader(dataset_test,batch_size=batch_size_test, shuffle=True,drop_last=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting Users\8kta\Documents\GitHub\deeplearning_project/MNIST/raw/train-images-idx3-ubyte.gz to Users\8kta\Documents\GitHub\deeplearning_project/MNIST/raw




Extracting Users\8kta\Documents\GitHub\deeplearning_project/MNIST/raw/train-labels-idx1-ubyte.gz to Users\8kta\Documents\GitHub\deeplearning_project/MNIST/raw




Extracting Users\8kta\Documents\GitHub\deeplearning_project/MNIST/raw/t10k-images-idx3-ubyte.gz to Users\8kta\Documents\GitHub\deeplearning_project/MNIST/raw




Extracting Users\8kta\Documents\GitHub\deeplearning_project/MNIST/raw/t10k-labels-idx1-ubyte.gz to Users\8kta\Documents\GitHub\deeplearning_project/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
shuffled_train_loader = dataset.SuffleMNIST(train_loader, num=4, radius = 42, wall_shape = 112, sum = True)
shuffled_val_loader = dataset.SuffleMNIST(val_loader, num=4, radius = 42, wall_shape = 112, sum = True)
shuffled_test_loader = dataset.SuffleMNIST(test_loader, num=4, radius = 42, wall_shape = 112, sum = True)

In [ ]:
print('There are {} images and {} labels in the train set.'.format(len(shuffled_train_loader.set_img),len(shuffled_train_loader.set_label)))
print('There are {} images and {} labels in the train set.'.format(len(shuffled_val_loader.set_img),len(shuffled_val_loader.set_label)))
print('There are {} images and {} labels in the train set.'.format(len(shuffled_test_loader.set_img),len(shuffled_test_loader.set_label)))

In [ ]:
trainshuffled_loader = torch.utils.data.DataLoader(shuffled_train_loader, batch_size=batch_size_train, shuffle=True,drop_last=False)
valshuffled_loader = torch.utils.data.DataLoader(shuffled_val_loader, batch_size=batch_size_train, shuffle=True,drop_last=False)
testshuffled_loader = torch.utils.data.DataLoader(shuffled_test_loader, batch_size=batch_size_test, shuffle=True,drop_last=False)

# Construcción de CNN

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Instantiate two convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=20, kernel_size=3, padding=1)
        
        # Instantiate the ReLU nonlinearity
        self.relu = nn.ReLU(inplace = True)
        
        # Instantiate a max pooling layer
        self.pool = nn.MaxPool2d(kernel_size= 2, stride =2 )
        
        # Instantiate a fully connected layer (python starts with zero!!)
        self.fc = nn.Linear(28*28*20 , 37)

    def forward(self, x):

        # Apply conv followd by relu, then in next line pool
        x = self.relu(self.conv1(x))
        x = self.pool(x)

        # Apply conv followd by relu, then in next line pool
        x = self.relu(self.conv2(x))
        x = self.pool(x)

        # Prepare the image for the fully connected layer
        x = x.view(-1, 28*28*20)

        # Apply the fully connected layer and return the result
        return self.fc(x)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Net().parameters(), lr = 3e-4)
model = Net()
model.cuda()

In [9]:
#model.train()
for epoch in range(1):
  for i, data in enumerate(trainshuffled_loader, 0):
      inputs, labels = data
      optimizer.zero_grad()


      #Give format to vector
      inputs = torch.unsqueeze(inputs,1) 

      # Compute the forward pass
      outputs = model(inputs)
          
      # Compute the loss function
      loss = criterion(outputs,labels)
          
      # Compute the gradients
      loss.backward()
      
      # Update the weights
      optimizer.step()

print('Finished Training')      

KeyboardInterrupt: ignored

In [ ]:
correct, total = 0,0
predictions = []
model.eval()

# Iterate over the data in the valshuffled_loader
for i, data in enumerate(valshuffled_loader,0):
  
    # Get the image and label from data
    image, label = data

    #Give format to vector
    image = torch.unsqueeze(image,1) 
    
    # Make a forward pass in the net with your image
    output = model(image)
    
    # Argmax the results of the net
    _, predicted = torch.max(output.data, 1)
    predictions.append(output)

    total += label.size(0)
    correct += (predicted == label).sum().item()

print('The validation set accuracy of network is : {} %%'.fotmat((100*correct / total)))

   # if predicted == label:
   #     print("Yipes, your net made the right prediction " + str(predicted))
   # else:
   #     print("Your net prediction was " + str(predicted) + ", but the correct label is: " + str(label))

In [16]:
for i, data in enumerate(trainshuffled_loader, 0):
    inputs, labels = data
    optimizer.zero_grad()

    # Compute the forward pass
    outputs = Net(inputs)
        
    # Compute the loss function
    loss = criterion(outputs,labels)
        
    # Compute the gradients
    loss.backward()
    
    # Update the weights
    optimizer.step()

NameError: ignored

In [71]:
url = 'https://raw.githubusercontent.com/8kta/deeplearning_project/main/core/resnet.py'
r = requests.get(url)

# make sure your filename is the same as how you want to import 
with open('resnet.py', 'w') as f:
    f.write(r.text)

# now we can import
import resnet

In [72]:
net = resnet.resnet18()

In [73]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 3e-4)

In [74]:
#hacemos un iterable con los elementos en test_loader
examples_train = enumerate(trainshuffled_loader)
#elegimos solamente uno de los i terables (el siguiente)
batch_idx, (example_data, example_targets) = next(examples_train)

In [75]:
examples_train = enumerate(trainshuffled_loader)
for epoch in range(10):
  count, inputs = next(examples_train)

  optimizer.zero_grad()

  inputs = inputs[0]

  inputs = torch.unsqueeze(inputs,1) 
  outputs = net(inputs)
  #outputs = AlexNet(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()

  # print statistics
  running_loss += loss.item()
  if i % 2000 == 1999:    # print every 2000 mini-batches
      print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss / 2000))
      running_loss = 0.0

print('Finished Training')

RuntimeError: ignored